# CRIM Intervals Demo notebook

This notebook illustrates the use of the new methods `getSoundingCount`, `getMeasure`, and `getTimeSignature`.

In [1]:
%cd ../intervals
from main import *
import pandas as pd
import music21

/Users/dangtrang/OneDrive - brynmawr.edu/summer 2021/crim_intervals/intervals


In [2]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0013.mei'])
model = corpus.scores[0]
ts = model.getTimeSignature()
ms = model.getMeasure()
sc = model.getSoundingCount()
nr = model.getNoteRest()

Requesting file from https://crimproject.org/mei/CRIM_Model_0013.mei...
Successfully imported.


## `getMeasure` to look at notes and measure

In this example, we build a table of notes and measures, and extract notes from measure 22.

### In the first method, after combining notes and measures, we use dataframe condition to index into the rows with the correct measure

In [26]:
# combine [offsets, notes] and [offsets, measures]
nr_ms = pd.concat([nr, ms], axis=1)
nr_ms

,Superius,Contratenor,Tenor,Bassus,Superius,Contratenor,Tenor,Bassus
0.0,D5,G4,Rest,G3,1.0,1.0,1.0,1.0
2.0,D5,G4,NaN,G3,NaN,NaN,NaN,NaN
3.0,D5,G4,NaN,G3,NaN,NaN,NaN,NaN
4.0,D5,B-4,NaN,G3,NaN,NaN,NaN,NaN
6.0,D5,B-4,NaN,B-3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
426.0,G4,E4,B-3,G3,NaN,NaN,NaN,NaN
427.0,NaN,D4,NaN,NaN,NaN,NaN,NaN,NaN
428.0,G4,E4,C4,C3,NaN,NaN,NaN,NaN
430.0,F4,D4,A3,D3,NaN,NaN,NaN,NaN


The table has all of the notes and rests offsets, and measures offsets. 

**Unfortunately, we can only see the measure offsets whenever a new measure begins (and we have NaN in other cases). Therefore, we would use forward fill to propagate the measures offsets and give these notes measures.**

In [23]:
# propagate measures offsets forwards (until a new measure begins)
filled_ms = nr_ms.iloc[:, 4:].ffill() 

In [25]:
filled_ms

,Superius,Contratenor,Tenor,Bassus
0.0,1.0,1.0,1.0,1.0
2.0,1.0,1.0,1.0,1.0
3.0,1.0,1.0,1.0,1.0
4.0,1.0,1.0,1.0,1.0
6.0,1.0,1.0,1.0,1.0
...,...,...,...,...
426.0,54.0,54.0,54.0,54.0
427.0,54.0,54.0,54.0,54.0
428.0,54.0,54.0,54.0,54.0
430.0,54.0,54.0,54.0,54.0


**retrieve notes from a specific measure**

In [28]:
# find measure 22 in complete measure table
mask = filled_ms == 22

# retrieve the notes in measure 22
m22_nr = nr_ms.iloc[:, :4][mask].dropna(how='all')

In [29]:
m22_nr

,Superius,Contratenor,Tenor,Bassus
168.0,B-4,Rest,None,D3
169.0,NaN,NaN,F3,NaN
170.0,NaN,B-3,B-3,E3
172.0,Rest,E4,NaN,C3
174.0,C5,E4,A3,NaN


### We can also turn measures into the notes' index, and then just index directly into the table

In [47]:
# get 1 col of measures (in all voices, measures are expected to have the same offset) (?)
measures = ms.iloc[:, 0]
measures.name = 'Measure'
df = pd.concat([nr, measures], axis=1)
df.Measure.ffill(inplace=True)
# set the "Measure" column as the index
df = df.set_index('Measure')
# get measure slices with .loc, note that .loc slicing is inclusive of the end
mm22_24 = df.loc[22:24, :]

In [49]:
mm22_24

,Superius,Contratenor,Tenor,Bassus
Measure,,,,
22.0,B-4,Rest,None,D3
22.0,NaN,NaN,F3,NaN
22.0,NaN,B-3,B-3,E3
22.0,Rest,E4,NaN,C3
22.0,C5,E4,A3,NaN
23.0,D5,D4,B-3,B-2
23.0,NaN,E4,NaN,NaN
23.0,D5,F4,D4,NaN
23.0,NaN,E4,NaN,NaN


### Important

No matter what we choose to do, we have to remember to use ffil to give all notes measures.

## `getTimeSignture` with notes and measures

### notes with certain time signature

In [57]:
nr_ts = pd.concat([nr, ts], axis=1)
filled_ts = nr_ts.iloc[:, 4:].ffill() 
# retrieve all notes with 
nr_ts.iloc[:, :4][filled_ts == '4/2'].dropna(how='all')

,Superius,Contratenor,Tenor,Bassus
0.0,D5,G4,Rest,G3
2.0,D5,G4,NaN,G3
3.0,D5,G4,NaN,G3
4.0,D5,B-4,NaN,G3
6.0,D5,B-4,NaN,B-3
...,...,...,...,...
424.0,None,F4,None,F3
426.0,G4,E4,B-3,G3
427.0,NaN,D4,NaN,NaN
428.0,G4,E4,C4,C3


### measures with certain time signature

In [63]:
ms_ts = pd.concat([ms, ts], axis=1)
filled_ms_ts = ms_ts.iloc[:, 4:].ffill() 
ms_ts.iloc[:, :4][filled_ms_ts == '8/2'].dropna(how='all')

,Superius,Contratenor,Tenor,Bassus
432.0,55.0,55.0,55.0,55.0


## Sounding Count and Cadence

In [33]:
har = model.getHarmonic('d', True, False)
n3 = model.getNgrams(how='modules', df=har, cell_type=str)

In [34]:
#  list of n=3 cadences assuming the interval settings used above
cadences = ['7_Held, 6_-2, 1', '4_Held, 3_-5, 1', '4_Held, 3_4, 1']
# find out which cells satisfy a condition
condition = n3.isin(cadences)
# filter out the cadences using that condition
cadential = n3[condition].dropna(how='all')
# count each type of cadence from any voice pair
cad_frequency = cadential.stack().value_counts()

In [35]:
# combine cadential and soundingcount
sc = model.getSoundingCount()
rsc = sc.reindex_like(cadential) # match get sounding count's indices with cadential's indices
cad_sc = pd.concat([cadential, rsc], axis=1) # [offset, cadence] + [offset, sounding]
# how many cadences happen in each voice pair for a given number of sounding voices?
gb_sc = cad_sc.groupby('Sounding').count().T

In [46]:
gb_sc

Sounding,3,4
Contratenor_Superius,1,1
Tenor_Contratenor,0,0
Tenor_Superius,0,2
Bassus_Tenor,2,0
Bassus_Contratenor,0,0
Bassus_Superius,0,3
